## Bias correcton and CDD Over West-Africa

In [1]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns
import pandas as pd
import os
import sys
import xarray as xr
import xclim

/home/ubuntu/.local/lib/python3.6/site-packages/numba/core/errors.py:154: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
os.chdir('/mys3bucket/Biascorrected/') # Setting working directory

## Historical data

In [3]:
MME_ERA=xr.open_dataset('/mys3bucket/ERA_Temp/Daily_air_temperature_at_2_metres_1970_2020_WA.nc')

/home/ubuntu/.local/lib/python3.6/site-packages/xarray/coding/times.py:134: CFWarning: this date/calendar/year zero convention is not supported by CF
  cftime.num2date(num_dates, units, calendar, only_use_cftime_datetimes=True)
/home/ubuntu/.local/lib/python3.6/site-packages/xarray/coding/times.py:463: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)


In [4]:
MME_ERA

/home/ubuntu/.local/lib/python3.6/site-packages/xarray/coding/times.py:134: CFWarning: this date/calendar/year zero convention is not supported by CF
  cftime.num2date(num_dates, units, calendar, only_use_cftime_datetimes=True)
/home/ubuntu/.local/lib/python3.6/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


<xarray.Dataset>
Dimensions:    (bnds: 2, latitude: 121, longitude: 181, time: 18993)
Coordinates:
  * time       (time) datetime64[ns] 1970-01-01T11:30:00 ... 2020-12-31T23:00:00
  * longitude  (longitude) float32 -20.0 -19.75 -19.5 -19.25 ... 24.5 24.75 25.0
  * latitude   (latitude) float32 30.0 29.75 29.5 29.25 ... 0.75 0.5 0.25 0.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    t2m        (time, latitude, longitude) float32 ...
Attributes:
    CDI:          Climate Data Interface version ?? (http://mpimet.mpg.de/cdi)
    Conventions:  CF-1.6
    history:      Wed Jan 12 06:18:21 2022: cdo mergetime Daily_tas_era5_1970...
    frequency:    day
    CDO:          Climate Data Operators version 1.9.3 (http://mpimet.mpg.de/...

In [5]:
MME_ERA=MME_ERA.drop(['time_bnds'])

In [6]:
# MME_ERA

In [7]:
# Convert to DegC
MME_ERA_df=MME_ERA.t2m-273.15
# copy attributes to get nice figure labels and change Kelvin to Celsius
MME_ERA_df.attrs = MME_ERA.t2m.attrs
MME_ERA_df.attrs["units"] = "deg C"
MME_ERA_df=MME_ERA_df.rename({'latitude': 'lat','longitude': 'lon'})

In [8]:
MME_ERA_df

<xarray.DataArray 't2m' (time: 18993, lat: 121, lon: 181)>
array([[[19.547333 , 19.50534  , 19.513031 , ..., 12.208893 ,
         12.386322 , 12.46347  ],
        [19.660492 , 19.669067 , 19.675903 , ..., 12.050293 ,
         12.330597 , 12.450623 ],
        [19.810486 , 19.840485 , 19.847351 , ..., 11.746857 ,
         11.798279 , 11.878021 ],
        ...,
        [25.288818 , 25.23996  , 25.215118 , ..., 23.907898 ,
         23.641296 , 23.649017 ],
        [25.271667 , 25.221954 , 25.20224  , ..., 23.74588  ,
         23.491272 , 23.453583 ],
        [25.245972 , 25.214264 , 25.234833 , ..., 23.656738 ,
         23.432983 , 23.354126 ]],

       [[19.957916 , 19.945068 , 19.956207 , ..., 11.079956 ,
         11.30368  , 11.651703 ],
        [20.025635 , 20.023071 , 20.030792 , ..., 10.941956 ,
         11.271118 , 11.534271 ],
        [20.101074 , 20.112213 , 20.106232 , ..., 11.055115 ,
         11.002808 , 11.115112 ],
...
        [25.589691 , 25.571686 , 25.584564 , ..., 27.339233 ,
         26.748627 , 26.589203 ],
        [25.521973 , 25.496277 , 25.492828 , ..., 26.628632 ,
         26.26175  , 26.240326 ],
        [25.404541 , 25.378815 , 25.373688 , ..., 25.92572  ,
         25.894012 , 25.966858 ]],

       [[18.39441  , 18.503265 , 18.630981 , ..., 12.488342 ,
         12.542328 , 12.404327 ],
        [18.444122 , 18.514404 , 18.638702 , ..., 12.425751 ,
         12.5971985, 12.873199 ],
        [18.487854 , 18.582123 , 18.652435 , ..., 11.836853 ,
         12.027161 , 12.563751 ],
        ...,
        [25.66513  , 25.584564 , 25.532257 , ..., 27.277527 ,
         26.500885 , 26.086884 ],
        [25.680573 , 25.605133 , 25.547699 , ..., 26.560913 ,
         25.84256  , 25.49884  ],
        [25.639404 , 25.587128 , 25.514252 , ..., 25.988281 ,
         25.552826 , 25.240814 ]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1970-01-01T11:30:00 ... 2020-12-31T23:00:00
  * lon      (lon) float32 -20.0 -19.75 -19.5 -19.25 ... 24.25 24.5 24.75 25.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 1.0 0.75 0.5 0.25 0.0
Attributes:
    long_name:  2 metre temperature
    units:      deg C

In [9]:
ref=MME_ERA_df.sel(time=slice(None, "1990-12-31"))

In [10]:
ref

<xarray.DataArray 't2m' (time: 8035, lat: 121, lon: 181)>
array([[[19.547333, 19.50534 , 19.513031, ..., 12.208893, 12.386322,
         12.46347 ],
        [19.660492, 19.669067, 19.675903, ..., 12.050293, 12.330597,
         12.450623],
        [19.810486, 19.840485, 19.847351, ..., 11.746857, 11.798279,
         11.878021],
        ...,
        [25.288818, 25.23996 , 25.215118, ..., 23.907898, 23.641296,
         23.649017],
        [25.271667, 25.221954, 25.20224 , ..., 23.74588 , 23.491272,
         23.453583],
        [25.245972, 25.214264, 25.234833, ..., 23.656738, 23.432983,
         23.354126]],

       [[19.957916, 19.945068, 19.956207, ..., 11.079956, 11.30368 ,
         11.651703],
        [20.025635, 20.023071, 20.030792, ..., 10.941956, 11.271118,
         11.534271],
        [20.101074, 20.112213, 20.106232, ..., 11.055115, 11.002808,
         11.115112],
...
        [25.865723, 25.87085 , 25.90686 , ..., 24.31076 , 24.13675 ,
         24.021881],
        [25.920563, 25.956573, 26.00116 , ..., 23.894165, 23.706451,
         23.573578],
        [26.003723, 26.039734, 26.032013, ..., 23.529846, 23.362701,
         23.349854]],

       [[18.77414 , 18.743286, 18.733002, ..., 14.352722, 14.14444 ,
         13.865845],
        [18.808441, 18.790436, 18.769012, ..., 14.54303 , 14.373291,
         13.979858],
        [18.818726, 18.82129 , 18.763855, ..., 14.326172, 14.165009,
         13.821259],
        ...,
        [25.868286, 25.90686 , 25.936005, ..., 23.844452, 23.776733,
         23.50586 ],
        [25.774841, 25.818573, 25.834015, ..., 23.563293, 23.50586 ,
         23.30014 ],
        [25.688263, 25.730286, 25.735413, ..., 23.417572, 23.321564,
         23.178406]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1970-01-01T11:30:00 ... 1990-12-31T23:00:00
  * lon      (lon) float32 -20.0 -19.75 -19.5 -19.25 ... 24.25 24.5 24.75 25.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 1.0 0.75 0.5 0.25 0.0
Attributes:
    long_name:  2 metre temperature
    units:      deg C

In [11]:
#function that provides to run the bias correction through all files for R
def bias_process(ref,hist,model_name):
    
    # conversion in degC
    model_load=xr.open_dataset('/mys3bucket/NEX_FUT_DAT/'+model_name)
    # copy attributes to get nice figure labels and change Kelvin to Celsius
    model_load_df=model_load.tasmax-273.15
    model_load_df.attrs = model_load.tasmax.attrs
    model_load_df.attrs["units"] = "deg C"
    
    print('End load future data ')
    
    ## Import historical data
    
    # conversion in degC
    hist_df=xr.open_dataset('/mys3bucket/NEX_FUT_DAT/'+hist)
    
    # copy attributes to get nice figure labels and change Kelvin to Celsius
    hist_df_dt=hist_df.tasmax-273.15
    hist_df_dt.attrs = hist_df.tasmax.attrs
    hist_df_dt.attrs["units"] = "deg C"
    
    hist_df_train=hist_df_dt.sel(time=slice("1970-01-01", "1990-12-31"))
    hist_df_test=hist_df_dt.sel(time=slice("1991-01-01", "2005-12-31"))

    print('End historical data')
    
    split_name=model_name.split('.')
    outputfile='/mys3bucket/Biascorrected/'+split_name[0]+'_Bias-eqm.nc'
    
    split_name_hist=str(hist)[:-22]+'1991_2005'
    outputfile_hist='/mys3bucket/Biascorrected/'+split_name_hist+'_Bias-eqm.nc'
    
    print (hist_df_train.shape)
    
    print (model_load_df.shape)
    ## Bias correction
    
    from xclim import sdba
    EQM = sdba.adjustment.EmpiricalQuantileMapping()
    EQM.train(ref,hist_df_train)
    scen = EQM.adjust(model_load_df)
#     scen2 = EQM.adjust(hist_df_test)
    
    print (outputfile)
    scen.to_netcdf(outputfile)
    
#     print (outputfile_hist)
#     scen2.to_netcdf(outputfile_hist)
    

## Import future data

In [12]:
Models_rcp45=['tas_day_BCSD_rcp45_r1i1p1_CNRM-CM5_2006_2100_WAfrik_rg.nc',
'tas_day_BCSD_rcp45_r1i1p1_CSIRO-Mk3-6-0_2006_2100_WAfrik_rg.nc',
'tas_day_BCSD_rcp45_r1i1p1_GFDL-ESM2M_2006_2100_WAfrik_rg.nc',
'tas_day_BCSD_rcp45_r1i1p1_IPSL-CM5A-MR_2006_2100_WAfrik_rg.nc',
'tas_day_BCSD_rcp45_r1i1p1_MIROC5_2006_2099_WAfrik_rg.nc',
'tas_day_BCSD_rcp45_r1i1p1_MPI-ESM-LR_2006_2100_WAfrik_rg.nc',
'tas_day_BCSD_rcp45_r1i1p1_NorESM1-M_2006_2100_WAfrik_rg.nc']

In [13]:
Models_rcp85=['tas_day_BCSD_rcp85_r1i1p1_CNRM-CM5_2006_2100_WAfrik_rg.nc',
'tas_day_BCSD_rcp85_r1i1p1_CSIRO-Mk3-6-0_2006_2100_WAfrik_rg.nc',
'tas_day_BCSD_rcp85_r1i1p1_GFDL-ESM2M_2006_2100_WAfrik_rg.nc',
'tas_day_BCSD_rcp85_r1i1p1_IPSL-CM5A-MR_2006_2100_WAfrik_rg.nc',
'tas_day_BCSD_rcp85_r1i1p1_MIROC5_2006_2099_WAfrik_rg.nc',
'tas_day_BCSD_rcp85_r1i1p1_MPI-ESM-LR_2006_2100_WAfrik_rg.nc',
'tas_day_BCSD_rcp85_r1i1p1_NorESM1-M_2006_2100_WAfrik_rg.nc']

In [14]:
Model_hist=['tas_day_BCSD_historical_r1i1p1_CNRM-CM5_1950_2005_WAfrik_rg.nc',
'tas_day_BCSD_historical_r1i1p1_CSIRO-Mk3-6-0_1950_2005_WAfrik_rg.nc',
'tas_day_BCSD_historical_r1i1p1_GFDL-ESM2M_1950_2005_WAfrik_rg.nc',
'tas_day_BCSD_historical_r1i1p1_IPSL-CM5A-MR_1950_2005_WAfrik_rg.nc',
'tas_day_BCSD_historical_r1i1p1_MIROC5_1950_2005_WAfrik_rg.nc',
'tas_day_BCSD_historical_r1i1p1_MPI-ESM-LR_1950_2005_WAfrik_rg.nc',
'tas_day_BCSD_historical_r1i1p1_NorESM1-M_1950_2005_WAfrik_rg.nc'
]

## Perform bias correction

In [15]:
#### RCP 85

In [16]:
# for i,model_name in enumerate(Models_rcp85):
    
#     hist=Model_hist[i]
#     print(hist)
    
# #     print('/mys3bucket/NEX_FUT_DAT/'+hist)
#     bias_process(ref,hist,model_name)

In [17]:
#### RCP 45

In [18]:
# for i,model_name in enumerate(Models_rcp45):
    
#     hist=Model_hist[i]
#     print(hist)
    
#     print('/mys3bucket/NEX_FUT_DAT/'+hist)
#     bias_process(ref,hist,model_name)

In [19]:
Models_rcp85=['tas_day_BCSD_rcp85_r1i1p1_IPSL-CM5A-MR_2006_2100_WAfrik_rg.nc',
'tas_day_BCSD_rcp85_r1i1p1_MPI-ESM-LR_2006_2100_WAfrik_rg.nc']

In [20]:
Model_hist=[
'tas_day_BCSD_historical_r1i1p1_IPSL-CM5A-MR_1950_2005_WAfrik_rg.nc',
'tas_day_BCSD_historical_r1i1p1_MPI-ESM-LR_1950_2005_WAfrik_rg.nc',
]

In [21]:
#function that run bias correction through all files
def bias_process_2(ref,hist,model_name):
    
    # conversion in degC
    model_load=xr.open_dataset('/mys3bucket/NEX_FUT_DAT/'+model_name)
    # copy attributes to get nice figure labels and change Kelvin to Celsius
    model_load_df=model_load.tasmax-273.15
    model_load_df.attrs = model_load.tasmax.attrs
    model_load_df.attrs["units"] = "deg C"
    
    print('End load future data ')
    
    ## Import historical data
    
    # conversion in degC
    hist_df=xr.open_dataset('/mys3bucket/NEX_FUT_DAT/'+hist)
    
    # copy attributes to get nice figure labels and change Kelvin to Celsius
    hist_df_dt=hist_df.tasmax-273.15
    hist_df_dt.attrs = hist_df.tasmax.attrs
    hist_df_dt.attrs["units"] = "deg C"
    
    hist_df_train=hist_df_dt.sel(time=slice("1970-01-01", "1990-12-31"))
    hist_df_test=hist_df_dt.sel(time=slice("1991-01-01", "2005-12-31"))

    print('End historical data')
    
    split_name=model_name.split('.')
    outputfile='/mys3bucket/Biascorrected/'+split_name[0]+'_Bias-eqm.nc'
    
    split_name_hist=str(hist)[:-22]+'1991_2005'
    outputfile_hist='/mys3bucket/Biascorrected/'+split_name_hist+'_Bias-eqm.nc'
    
    print (hist_df_train.shape)
    
    print (model_load_df.shape)
    ## Bias correction
    
    from xclim import sdba
    EQM = sdba.adjustment.EmpiricalQuantileMapping()
    EQM.train(ref,hist_df_train)
#     scen = EQM.adjust(model_load_df)
    scen2 = EQM.adjust(hist_df_test)
    
#     print (outputfile)
#     scen.to_netcdf(outputfile)
    
    print (outputfile_hist)
    scen2.to_netcdf(outputfile_hist)
    

In [22]:
# for i,model_name in enumerate(Models_rcp85):
    
#     hist=Model_hist[i]
#     print(hist)
    
# #     print('/mys3bucket/NEX_FUT_DAT/'+hist)
#     bias_process_2(ref,hist,model_name)

## Compute Cooling Degree Days

In [23]:
from glob import glob

In [24]:
## Get all bias corrected files
files_bias=glob('/mys3bucket/Biascorrected/*_1991_2005_Bias-eqm.nc') # list of cordex

In [25]:
files_bias

['/mys3bucket/Biascorrected/tas_day_BCSD_historical_r1i1p1_IPSL-CM5A-MR_1991_2005_Bias-eqm.nc',
 '/mys3bucket/Biascorrected/tas_day_BCSD_historical_r1i1p1_MPI-ESM-LR_1991_2005_Bias-eqm.nc']

In [26]:
#function that run the cdd through all files 
def cdd_process(file):
    print (file)
    dat_bias=xr.open_dataarray(file)
    
    # Set unit back again to Kelvin for the package xclim
    dat_bias=dat_bias+273.15
    dat_bias.attrs["units"] = "K"
    
    cdd = xclim.atmos.cooling_degree_days(tas=dat_bias, thresh="25.0 degC", freq="YS")
    
    split_name=file.split('.')
    outputfile='/mys3bucket/cdd_output/CDD/CDD_'+file.split('/')[3]
    
    print (outputfile)
    cdd.to_netcdf(outputfile)

In [34]:
for file in files_bias: 
    cdd_process(file)

In [35]:
# RCP4.5 
## Get all bias corrected files
files_bias=glob('/mys3bucket/Biascorrected/tas_day_BCSD_rcp45_r1i1p1*_Bias-eqm.nc') # list of files
files_bias

In [32]:
for file in files_bias: 
    cdd_process(file)

In [30]:
# RCP8.5 
## Get all bias corrected files
files_bias=glob('/mys3bucket/Biascorrected/tas_day_BCSD_rcp85_r1i1p1*_Bias-eqm.nc') # list of files
files_bias

['/mys3bucket/Biascorrected/tas_day_BCSD_rcp85_r1i1p1_CNRM-CM5_2006_2100_WAfrik_rg_Bias-eqm.nc',
 '/mys3bucket/Biascorrected/tas_day_BCSD_rcp85_r1i1p1_CSIRO-Mk3-6-0_2006_2100_WAfrik_rg_Bias-eqm.nc',
 '/mys3bucket/Biascorrected/tas_day_BCSD_rcp85_r1i1p1_GFDL-ESM2M_2006_2100_WAfrik_rg_Bias-eqm.nc',
 '/mys3bucket/Biascorrected/tas_day_BCSD_rcp85_r1i1p1_IPSL-CM5A-MR_1991_2100_WAfrik_rg_Bias-eqm.nc',
 '/mys3bucket/Biascorrected/tas_day_BCSD_rcp85_r1i1p1_IPSL-CM5A-MR_2006_2100_WAfrik_rg_Bias-eqm.nc',
 '/mys3bucket/Biascorrected/tas_day_BCSD_rcp85_r1i1p1_MIROC5_2006_2099_WAfrik_rg_Bias-eqm.nc',
 '/mys3bucket/Biascorrected/tas_day_BCSD_rcp85_r1i1p1_MPI-ESM-LR_1991_2100_WAfrik_rg_Bias-eqm.nc',
 '/mys3bucket/Biascorrected/tas_day_BCSD_rcp85_r1i1p1_MPI-ESM-LR_2006_2100_WAfrik_rg_Bias-eqm.nc',
 '/mys3bucket/Biascorrected/tas_day_BCSD_rcp85_r1i1p1_NorESM1-M_2006_2100_WAfrik_rg_Bias-eqm.nc']

In [33]:
for file in files_bias: 
    cdd_process(file)